# 2. Incoming Event Handler
  --------------------------------------------------------------------

Handle incoming events and write them to an output V3IO Stream `incoming-events-stream`.
The received data is partitioned by `user_id`.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

The rest of the notebooks rely on the output stream of this notebook. Therefore, one can change the input data without affecting the rest of the workflow.

## Create and Test a Local Function 
Import nuclio SDK and magics, <b>do not remove the cell and comment !!!</b>

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%run config.py

In [3]:
import os
os.environ["MDWS_EH_OUTPUT_STREAM_PATH"] = STREAM_CONFIGS['incoming-events-stream']['path']
os.environ["MDWS_EH_PARTITION_ATTR"] = PARTITION_ATTR
os.environ["MDWS_EH_INPUT_STREAM_URL"] = path.join(WEB_API_USERS, STREAM_CONFIGS['generated-stream']['path']) + "@incomingeventhandler"
os.environ["WEB_API"] = WEB_API

In [4]:
# nuclio: start-code

#### Functions imports

In [5]:
import v3io.dataplane

<b>Specify function dependencies and configuration<b>

In [6]:
%nuclio cmd -c pip install v3io

In [7]:
%%nuclio env
V3IO_ACCESS_KEY = ${V3IO_ACCESS_KEY}
CONTAINER = users
MDWS_EH_OUTPUT_STREAM_PATH = ${MDWS_EH_OUTPUT_STREAM_PATH}
PARTITION_ATTR = ${MDWS_EH_PARTITION_ATTR}
WEB_API = ${WEB_API}

%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'CONTAINER' environment variable
%nuclio: setting 'MDWS_EH_OUTPUT_STREAM_PATH' environment variable
%nuclio: setting 'PARTITION_ATTR' environment variable
%nuclio: setting 'WEB_API' environment variable


In [8]:
%%nuclio config
spec.triggers.v3io_stream.kind = "v3ioStream"
spec.triggers.v3io_stream.disabled = false
spec.triggers.v3io_stream.url = "${MDWS_EH_INPUT_STREAM_URL}"
spec.triggers.v3io_stream.maxWorkers = 10
spec.triggers.v3io_stream.password = "${V3IO_ACCESS_KEY}"
spec.triggers.v3io_stream.attributes.pollingIntervalMs = 500
spec.triggers.v3io_stream.attributes.seekTo = "earliest"
spec.triggers.v3io_stream.attributes.readBatchSize = 64

%nuclio: setting spec.triggers.v3io_stream.kind to 'v3ioStream'
%nuclio: setting spec.triggers.v3io_stream.disabled to False
%nuclio: setting spec.triggers.v3io_stream.url to 'http://v3io-webapi:8081/users/iguazio/examples/model-deployment-with-streaming/generated-stream@incomingeventhandler'
%nuclio: setting spec.triggers.v3io_stream.maxWorkers to 10
%nuclio: setting spec.triggers.v3io_stream.password to '4868d73d-1bd9-486e-91bd-d89ad83e3a56'
%nuclio: setting spec.triggers.v3io_stream.attributes.pollingIntervalMs to 500
%nuclio: setting spec.triggers.v3io_stream.attributes.seekTo to 'earliest'
%nuclio: setting spec.triggers.v3io_stream.attributes.readBatchSize to 64


## Function code

In [9]:
import os
import json
def init_context(context):
    V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
    CONTAINER = os.getenv('CONTAINER')
    OUTPUT_STREAM_PATH = os.getenv('MDWS_EH_OUTPUT_STREAM_PATH')
    PARTITION_ATTR = os.getenv('MDWS_PARTITION_ATTR')
    WEB_API = os.getenv('WEB_API')
    v3io_client = v3io.dataplane.Client(endpoint=WEB_API, access_key=V3IO_ACCESS_KEY)

    setattr(context, 'v3io_client', v3io_client)
    setattr(context, 'partition_attr', PARTITION_ATTR)
    setattr(context, 'container', CONTAINER)
    setattr(context, 'output_stream_path', OUTPUT_STREAM_PATH)


def handler(context, event):
    print(dir(context))
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    context.logger.info_with('Got invoked',
                             trigger_kind=event.trigger.kind,
                             event_body=event_dict)
        
    partition_key = event_dict.get(context.partition_attr)
    record = event_to_record(event_dict, partition_key)
    
    print("saasasas" + context.output_stream_path)
    resp = context.v3io_client.put_records(container=context.container, 
                                   path=context.output_stream_path, 
                                   records=[record], 
                                   raise_for_status=v3io.dataplane.RaiseForStatus.never)
    
    context.logger.info_with('Sent event to stream', 
                             record=record,
                             response_status=resp.status_code, 
                             response_body=resp.body.decode('utf-8'))
    
    return resp.status_code


def event_to_record(event_dict, partition_key):
    event_str = json.dumps(event_dict)
    return {'data': event_str, 'partition_key': str(partition_key)}

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [10]:
# nuclio: end-code
# marks the end of a code section

## Test locally

In [11]:
event = nuclio.Event(body=b'{"user_id" : 111111 , "event_type": "spin"}')
init_context(context)
handler(context, event)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'container', 'logger', 'output_stream_path', 'partition_attr', 'set_logger_level', 'trigger_name', 'user_data', 'v3io_client', 'worker_id']
Python> 2020-08-04 22:00:44,317 [info] Got invoked: {'trigger_kind': '', 'event_body': {'user_id': 111111, 'event_type': 'spin'}}
saasasasiguazio/examples/model-deployment-with-streaming/incoming-events-stream
Python> 2020-08-04 22:00:44,319 [info] Sent event to stream: {'record': {'data': '{"user_id": 111111, "event_type": "spin"}', 'partition_key': 'None'}, 'response_status': 200, 'response_body': '{ "FailedRecordCount":0,"Records": [{ "SequenceNumber":1,"ShardId":3 } ] }'}


200

## Deploy function

In [12]:
%nuclio deploy -p {PROJECT_NAME} -n {V3IO_USERNAME}-incoming-event-handler

[nuclio] 2020-08-04 22:00:45,460 project name not found created new (model-deployment-with-streaming)
[nuclio] 2020-08-04 22:00:51,580 (info) Build complete
[nuclio] 2020-08-04 22:00:55,629 (info) Function deploy complete
[nuclio] 2020-08-04 22:00:55,635 done creating iguazio-incoming-event-handler, function address: 3.14.105.22:32190
%nuclio: function deployed
